Import required libraries

In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import warnings
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

Load the dataset from CSV

In [2]:
df = pd.read_csv('./data/initial_eda.csv')

In [3]:
df.head()

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count
0,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",1547656134,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",258,51
1,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",nathanwj,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,1547658625,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,141,27
2,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Rowyfo,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",1547662874,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",168,31
3,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"That sounds a lot like what's going on, thanks!",1547663080,"That sounds a lot like what's going on, thanks!",47,9
4,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",nongaussian,I will probably stick to GoodNotes 4 for a while. Noticed that in GoodNotes 5 there is no easy way to export all your notes to Dropbox in PDF format. Love to have those available for my non-Apple devices (i.e. most of them).\n\nNot a complaint: I expect that GN 5 will mature and g

Take a look at replies containing hyperlinks

In [4]:
df[df['reply'].str.contains(pat='.*https?://.*', regex=True, na=False)].head()

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count
9,agoowm,1,The bundle is available !,NaN,NaN,TabulatorSpalte,"Unfortunately, they are. http://imgur.com/UJqsKhy",1547717530,"Unfortunately, they are. http://imgur.com/UJqsKhy",49,4
17,agpzxb,1,What happened to the pen (Goodnotes 5)?,NaN,"I just got Goodnotes 5 and I was so excited for vertical scrolling only to be completely disappointed by the inking behaviour. I had previously switched from Notability because I didn’t like their inking behaviour... and now the pen in Goodnotes 5 feels just like Notability’s. Am I the only one who’s feeling this way?\n\nI’m not sure exactly what is wrong but I think it might be because there is too much variation in stroke thickness with pressure. I don’t want to completely disable the effects of pressure, but right now, the current fountain pen is just horrible. I always used the 1.9 unit thickness in Goodnotes 4, but in Goodnotes 5 the units of thickness has changed and I can not find a thickness that is equivalent to the old pen (maybe between the new 0.3 and 0.4 mm). \n\nCould we please have the option to revert to the old inking behaviour? :( for once I am glad that they released a completely new app because I can still use Goodnotes 4, but I have been waiting so long for vertical scrolling.",scykei,"Just for reference, this is how I usually write:\n\nhttps://imgur.com/a/De23nPy\n\nMy handwriting is quite small and I tend to fit a lot into a single line in split screen. Disabling pressure variation makes it hideous, and the new pens even more so. I am really disappointed at the moment.",1547706361,"Just for reference, this is how I usually write:\n\nhttps://imgur.com/a/De23nPy\n\nMy handwriting is quite small and I tend to fit a lot into a single line in split screen. Disabling pressure variation makes it hideous, and the new pens even more so. I am really disappointed at the moment.",286,49
18,agpzxb,1,What happened to the pen (Goodnotes 5)?,NaN,"I just got Goodnotes 5 and I was so excited for vertical scrolling only to be completely disappointed by the inking behaviour. I had previously switched from Notability because I didn’t like their inking behaviour... and now the pen in Goodnotes 5 feels just like Notability’s. Am I the only one who’s feeling this way?\n\nI’m not sure exactly what is wrong but I think it might be because there is too much variation in stroke thickness with pressure. I don’t want to completely disable the effects of pressure, but right now, the current fountain pen is just horrible. I always used the 1.9 unit thickness in Goodnotes 4, but in Goodnotes 5 the units of thickness has changed and I can not find a thickness that is equivalent to the old pen (maybe between the new 0.3 and 0.4 mm). \n\nCould we please have the option to revert to the old inking behaviour? :( for once I am glad that they released a completely new app because I can still use Goodnotes 4, but I have been waiting so long for vertical scrolling.",imguralbumbot,"^(Hi, I'm a bot for linking direct images of albums with only 1 image)\n\n**https://i.imgur.com/EwW0gp6.png**\n\n^^[Source](https://github.com/AUTplayed/imguralbumbot) ^^| ^^[Why?](https://github.com/AUTplayed/imguralbumbot/blob/master/README.md) ^^| ^^[Creator](https://np.reddit.com/user/AUTplayed/) ^^| ^^[ignoreme](https://np.reddit.com/message/compose/?to=imguralbumbot&amp;subject=ignoreme&amp;message=ignoreme) ^^| ^^[deletthis](https://np.reddit.com/message/compose/?to=imguralbumbot&amp;subject=delet%20this&amp;message=delet%20this%20ee98pl2)",1547706369,"^(Hi, I'm a bot for linking direct images of albums with only 1 image)\n\n**https://i.imgur.com/EwW0gp6.png**\n\n^^[Source](https://github.com/AUTplayed/imguralbumbot) ^^| ^^[Why?](https://github.com/AUTplayed/imguralbumbot/blob/master/README.md) ^^| ^^[Creator](https://np.reddit.com/user/AUTplayed/) ^^| ^

In [5]:
df[df['reply'].str.contains(pat='.*https?://.*', regex=True)].shape

(315, 11)

Remove hyperlinks from replies

In [6]:
df['reply_clean'] = df['reply'].str.replace(pat='[(]?https?://[A-Za-z0-9-_=/.?%&$;:,"}{ ]+[)]?', repl='', case=False, regex=True)

In [7]:
# Sanitary checks
df['reply_clean'][43] #17

'According to the [review at Macstories], it looks like you’re out of luck unfortunately.'

In [8]:
df['reply_clean'][17]

'Just for reference, this is how I usually write:\n\n\n\nMy handwriting is quite small and I tend to fit a lot into a single line in split screen. Disabling pressure variation makes it hideous, and the new pens even more so. I am really disappointed at the moment.'

In [9]:
# Sanitary checks
# df['submission_title'][2266]

In [10]:
# df[df['submission_id'] == 'ii5wzz']

Verify all hyperlinks have been removed

In [11]:
df[df['reply_clean'].str.contains(pat='.*https?://.*', regex=True, na=False)]

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count,reply_clean


In [12]:
# df_clean = df[~((df['submission_title'].str.contains(pat='.*https?://.*', regex=True, na=False)) | (df['submission_selftext'].str.contains(pat='.*https?://.*', regex=True, na=False)) | (df['reply'].str.contains(pat='.*https?://.*', regex=True, na=False)))]

Check for hexadecimal codes in messages

In [13]:
df[df['reply_clean'].str.contains(pat=':? #?([A-Fa-f0-9]){6}\\b', regex=True, na=False)]

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count,reply_clean
660,e2m2ao,1,Some suggestions and improvements for Goodnotes 5,NaN,"Hi there\n\nI have had GN5 installed on my iPad for a while now, and have come across some things that could be improved\n\n1) The app seems kind of slow and laggy when scrolling through documents, especially if there is a fair bit written on a page\n\n2) Writing occasionally disappears and then reappears\n\n3) The colour code doesn't show the one of the colour currently selected. It always shows FFEEFF\n\n4) Squared paper isn't sized how it should be compared to actual paper. On real paper you have squares that are 5mm wide and tall. The closest I have seen is the ""squared paper"" template, with 5,75mm intervals\n\n5) Pen sizes in GN4 and GN5 use different systems. A ""1"" in GN4 is approximately a ""0,2mm"" in GN5, while a text marker ""10"" in GN4 is a ""2mm"" in GN5. Am i correct in assuming a 1=0.2 ratio?\n\nI would be happy if someone were to look at these things and appreciate any response. Thank you.",dgaldeus,"1) I am using a 10.5 inch iPad Pro with everything updated to the current version, so I shouldn't have any issues regarding performance either. The blank pages happen to me as well, but sometimes when turning the page the app hangs for a bit before it fully changes to the next page. \n\n2) I will try to recreate that. \n\n3) Lets say I create a colour preset with the hex code 123456. When initially creating it, it shows me the correct code. But if I then want to change the colour it doesn't show me #123456 but #FFEEFF\n\n4) I didn't know that. But even then, If I use a small pen zoomed out it appears larger when zooming in. So shouldn't it be possible to create paper with 5mm intervals that stay at 5mm? If I use a marker with a with of 5mm it should always completely fill out a square, whether zommed in or not. After all that's how pens behave as well.\n\n5) In GN4, you don't have a unit when changing the pen size. You go from 0.5 to 10 in 0.1 step intervals. In GN5 you go from 0.1mm to 2mm in 0.05mm intervals.",1575125913,"1) I am using a 10.5 inch iPad Pro with everything updated to the current version, so I shouldn't have any issues regarding performance either. The blank pages happen to me as well, but sometimes when turning the page the app hangs for a bit before it fully changes to the next page. \n\n2) I will try to recreate that. \n\n3) Lets say I create a colour preset with the hex code 123456. When initially creating it, it shows me the correct code. But if I then want to change the colour it doesn't show me #123456 but #FFEEFF\n\n4) I didn't know that. But even then, If I use a small pen zoomed out it appears larger when zooming in. So shouldn't it be possible to create paper with 5mm intervals that stay at 5mm? If I use a marker with a with of 5mm it should always completely fill out a square, whether zommed in or not. After all that's how pens behave as well.\n\n5) In GN4, you don't have a unit when changing the pen size. You go from 0.5 to 10 in 0.1 step intervals. In GN5 you go from 0.1mm to 2mm in 0.05mm intervals.",1018,201,"1) I am using a 10.5 inch iPad Pro with everything updated to the current version, so I shouldn't have any issues regarding performance either. The blank pages happen to me as well, but sometimes when turning the page the app hangs for a bit before it fully changes to the next page. \n\n2) I will try to recreate that. \n\n3) Lets say I create a colour preset with the hex code 123456. When initially creating it, it shows me the correct code. But if I then want to change the colour it doesn't show me #123456 but #FFEEFF\n\n4) I didn't know that. But even then, If I use a small pen zoomed out it appears larger when zooming in. So shouldn't it be possible to create paper with 5mm intervals that stay at 5mm? If I use a marker

Remove all hexadecimal codes

In [14]:
df['reply_clean'] = df['reply_clean'].str.replace(pat=':? #?([A-Fa-f0-9]){6}\\b', repl='', case=False, regex=True)

In [15]:
# Sanitary check
# df['submission_title'][5766]

In [16]:
# Sanitary check
# df['submission_selftext'][656]

In [17]:
# Sanitary check
df['reply_clean'][660]

"1) I am using a 10.5 inch iPad Pro with everything updated to the current version, so I shouldn't have any issues regarding performance either. The blank pages happen to me as well, but sometimes when turning the page the app hangs for a bit before it fully changes to the next page. \n\n2) I will try to recreate that. \n\n3) Lets say I create a colour preset with the hex code. When initially creating it, it shows me the correct code. But if I then want to change the colour it doesn't show me but\n\n4) I didn't know that. But even then, If I use a small pen zoomed out it appears larger when zooming in. So shouldn't it be possible to create paper with 5mm intervals that stay at 5mm? If I use a marker with a with of 5mm it should always completely fill out a square, whether zommed in or not. After all that's how pens behave as well.\n\n5) In GN4, you don't have a unit when changing the pen size. You go from 0.5 to 10 in 0.1 step intervals. In GN5 you go from 0.1mm to 2mm in 0.05mm interv

Remove punctuation from messages

In [18]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


Convert punctuation characters to a list and add a custom apostrophe character to it

In [19]:
punctuation_list = list(string.punctuation)
punctuation_list.append('’')

In [20]:
df['reply_clean'] = df['reply_clean'].apply(lambda x: "".join([char.lower() for char in x if char not in punctuation_list]))

In [21]:
# df['reply_clean'] = df['reply_clean'].str.replace(pat='['+string.punctuation+'’'+']', repl=' ', regex=True)

Remove the newline character `\n` from messages

In [22]:
df['reply_clean'] = df['reply_clean'].str.replace(pat='\\n', repl=' ', regex=True)

Check dataframe to verify punctuation and newline character removal

In [23]:
df.head()

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count,reply_clean
0,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",1547656134,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",258,51,im getting a ton of bugs with 5 as well snappy lines no response at times and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone i think they need some time to cope with the new launch
1,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",nathanwj,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,1547658625,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,141,27,goodnotes 5 is not yet compatible with the desktop app as it says in the release notes there will be more features added in the near future
2,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Rowyfo,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",1547662874,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",168,31,i havent downloaded gn5 yet but watched a walkthrough and i did see that theres a snap option to check and uncheck i think somewhere in pen options hope that helps
3,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"That sounds a lot like what's going on, thanks!",1547663080,"That sounds a lot like what's going on, thanks!",47,9,that sounds a lot like whats going on thanks
4,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I

Convert all letters to lowercase

In [24]:
df['reply_clean'] = df['reply_clean'].str.lower()

Tokenize all words

In [25]:
df['reply_tokenized'] = df['reply_clean'].apply(lambda x: word_tokenize(x))

Remove stop words

In [26]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [27]:
df['reply_tokenized'] = df['reply_tokenized'].apply(lambda x: [word for word in x if word not in stop_words])

In [28]:
# for stop_word in stop_words:
#     df['reply'] = df['reply'].str.replace(pat='\\b'+stop_word+'\\b', repl=' ', regex=True)

Check if stop words have been removed

In [29]:
df.head()

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count,reply_clean,reply_tokenized
0,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",1547656134,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",258,51,im getting a ton of bugs with 5 as well snappy lines no response at times and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone i think they need some time to cope with the new launch,"[im, getting, ton, bugs, 5, well, snappy, lines, response, times, reason, ability, sync, google, drive, download, multiple, files, google, drive, gone, think, need, time, cope, new, launch]"
1,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",nathanwj,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,1547658625,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,141,27,goodnotes 5 is not yet compatible with the desktop app as it says in the release notes there will be more features added in the near future,"[goodnotes, 5, yet, compatible, desktop, app, says, release, notes, features, added, near, future]"
2,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Rowyfo,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",1547662874,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",168,31,i havent downloaded gn5 yet but watched a walkthrough and i did see that theres a snap option to check and uncheck i think somewhere in pen options hope that helps,"[havent, downloaded, gn5, yet, watched, walkthrough, see, theres, snap, option, check, uncheck, think, somewhere, pen, options, hope, helps]"
3,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the deskto

Stem the words

In [30]:
stemmer = PorterStemmer()
df['reply_preprocessed'] = df['reply_tokenized'].apply(lambda x: [stemmer.stem(word) for word in x])

In [31]:
# df['submission_selftext_stem'] = df['submission_selftext'].apply(lambda x: [stemmer.stem(word) for word in word_tokenize(x)] if x is not np.nan else [])

In [32]:
df.head()

,submission_id,submission_score,submission_title,submission_link_flair_text,submission_selftext,reply_author,reply_body,reply_created_utc,reply,reply_char_count,reply_word_count,reply_clean,reply_tokenized,reply_preprocessed
0,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Mauri97,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",1547656134,"I'm getting a ton of bugs with 5 as well (snappy lines, no response at times) and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone.\n\n I think they need some time to cope with the new launch.",258,51,im getting a ton of bugs with 5 as well snappy lines no response at times and for some reason the ability to sync to google drive and to download multiple files at once from google drive is gone i think they need some time to cope with the new launch,"[im, getting, ton, bugs, 5, well, snappy, lines, response, times, reason, ability, sync, google, drive, download, multiple, files, google, drive, gone, think, need, time, cope, new, launch]","[im, get, ton, bug, 5, well, snappi, line, respons, time, reason, abil, sync, googl, drive, download, multipl, file, googl, drive, gone, think, need, time, cope, new, launch]"
1,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",nathanwj,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,1547658625,Goodnotes 5 is not yet compatible with the desktop app as it says in the release notes. There will be more features added in the near future.,141,27,goodnotes 5 is not yet compatible with the desktop app as it says in the release notes there will be more features added in the near future,"[goodnotes, 5, yet, compatible, desktop, app, says, release, notes, features, added, near, future]","[goodnot, 5, yet, compat, desktop, app, say, releas, note, featur, ad, near, futur]"
2,aglcrj,1,Goodnotes 4 vs. Goodnotes 5 right now,NaN,"I have used Goodnotes 4 for work a ton. And I do mean a ton. I bought and downloaded 5 yesterday, transfer was easy. I realized I can't use the Mac App with 5. Right? Changes I made in 5 aren't synced to the desktop app. Also I kept getting a syncing error in 5. What was it syncing with? I am going to continue to play with it but I am not sure I feel comfortable diving in yet.",Rowyfo,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",1547662874,"I haven't downloaded GN5 yet but watched a walkthrough and I did see that there's a snap option to check and uncheck, I think somewhere in pen options. Hope that helps!",168,31,i havent downloaded gn5 yet but watched a walkthrough and i did see that theres a snap option to check and uncheck i think somewhere in pen options hope that helps,"[havent, downloaded, gn5, yet, watched, walkthrough, see, theres, snap, option, check, uncheck, think, somewhere, pen, options, 

Remove intermediate column

In [33]:
df = df.drop(columns=['reply_clean']).reset_index(drop=True)

Write dataframe to CSV

In [34]:
df.to_csv('./data/preprocessed.csv', index=False)